<a href="https://colab.research.google.com/github/kota-mizu/atmaCup-20/blob/main/notebooks/BaseLine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# !pip install ydata-profiling

In [4]:
# ライブラリのインポート
import os, warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from IPython.display import display
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error, r2_score
# from ydata_profiling import ProfileReport



warnings.filterwarnings("ignore")

# 日本語フォントを簡単に使う
!pip -q install japanize-matplotlib
import japanize_matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 44.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
data_path = '/content/drive/MyDrive/atmaCup#20/'

In [18]:
train_df  = pd.read_csv(data_path + 'train.csv')
test_df   = pd.read_csv(data_path + 'test.csv')
submission_df = pd.read_csv(data_path + 'sample_submission.csv')  # header 無し
career_df = pd.read_csv(data_path + 'career.csv')
dx_df = pd.read_csv(data_path + 'dx.csv')
hr_df = pd.read_csv(data_path + 'hr.csv')
overtime_df = pd.read_csv(data_path + 'overtime_work_by_month.csv')
position_df = pd.read_csv(data_path + 'position_history.csv')
udemy_activity_df = pd.read_csv(data_path + 'udemy_activity.csv')

In [8]:
display(udemy_activity_df)

,コースID,コースタイトル,レクチャーもしくはクイズ,レクチャー/クイズID,レクチャー/クイズの題名,開始日,終了日,推定完了率%,最終結果（クイズの場合）,マーク済み修了,コースカテゴリー
社員番号,,,,,,,,,,,
-1sqs0GXzpPJuAVKHUUFgg==,4615016,企業オリジナル講座,Quiz,5528090,企業オリジナル講座,2022/4/11 10:10,2022/4/11 10:10,100.0,0.0,True,企業オリジナル講座
-1sqs0GXzpPJuAVKHUUFgg==,4615016,企業オリジナル講座,Quiz,5528090,企業オリジナル講座,2022/4/11 10:11,2022/4/11 10:11,100.0,100.0,True,企業オリジナル講座
-1sqs0GXzpPJuAVKHUUFgg==,4615016,企業オリジナル講座,Quiz,5528100,企業オリジナル講座,2022/4/11 10:26,2022/4/11 10:26,100.0,0.0,True,企業オリジナル講座
-1sqs0GXzpPJuAVKHUUFgg==,4615016,企業オリジナル講座,Quiz,5528100,企業オリジナル講座,2022/4/11 10:27,2022/4/11 10:27,100.0,100.0,True,企業オリジナル講座
-1sqs0GXzpPJuAVKHUUFgg==,4615016,企業オリジナル講座,Quiz,5528102,企業オリジナル講座,2022/4/11 10:21,2022/4/11 10:21,100.0,100.0,True,企業オリジナル講座
...,...,...,...,...,...,...,...,...,...,...,...
zxY0Eflwm1tYj1Wt6vo_1g==,5264112,企業オリジナル講座,Video_lecture,37311144,企業オリジナル講座,2023/6/14 9:56,2023/6/14 9:56,100.0,NaN,True,企業オリジナル講座
zxY0Eflwm1tYj1Wt6vo_1g==,5264112,企業オリジナル講座,Video_lecture,37311150,企業オリジナル講座,2023/6/14 9:57,2023/6/14 9:57,100.0,NaN,True,企業オリジナル講座
zxY0Eflwm1tYj1Wt6vo_1g==,6098315,企業オリジナル講座,Video_lecture,45012691,企業オリジナル講座,2024/9/27 15:39,2024/9/27 16:17,100.0,NaN,True,企業オリジナル講座


In [31]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import GroupKFold
from sklearn.metrics import roc_auc_score

# 2. 特徴量の作成（最小限）
print("特徴量の作成を開始...")
# 2-1. 最新の役職と勤務区分を取得
# 年でソートし、最新のものを取得
latest_position = position_df.sort_values('year').drop_duplicates('社員番号', keep='last').drop(columns='year')

# 2-2. 平均残業時間を計算
avg_overtime = overtime_df.groupby('社員番号')['hours'].mean().reset_index().rename(columns={'hours': 'avg_overtime'})

# 3. データをマージ
print("データのマージを開始...")
# trainとtestを一度結合して、特徴量をまとめて付与する
train_df['is_train'] = 1
test_df['is_train'] = 0
all_df = pd.concat([train_df, test_df], ignore_index=True)

# 特徴量をマージ
all_df = pd.merge(all_df, latest_position, on='社員番号', how='left')
all_df = pd.merge(all_df, avg_overtime, on='社員番号', how='left')
print("データのマージ完了。")

# 4. 前処理
print("前処理を開始...")
# カテゴリカル変数をPandasのcategory型に変換
categorical_features = ['category', '勤務区分', '役職']
for col in categorical_features:
    all_df[col] = all_df[col].astype('category')

# 欠損値を0で埋める（暫定的な対応）
all_df['avg_overtime'] = all_df['avg_overtime'].fillna(0)

all_df = all_df.rename(columns = {"category" : "apply_category"})

# 再びtrainとtestに分割
train_processed = all_df[all_df['is_train'] == 1].drop(columns='is_train')
test_processed = all_df[all_df['is_train'] == 0].drop(columns=['is_train', 'target'])

# モデル学習用のデータを準備
X = train_processed.drop(columns=['target', '社員番号'])
y = train_processed['target']
X_test = test_processed.drop(columns='社員番号')
groups = train_processed['社員番号']
print("前処理完了。")

# 5. モデル学習と予測
print("モデル学習を開始...")
params = {
    'objective': 'binary',
    'metric': 'auc',
    'n_estimators': 1000,
    'learning_rate': 0.05,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 1,
    'verbose': -1,
    'n_jobs': -1,
    'seed': 42,
}

N_SPLITS = 5
gkf = GroupKFold(n_splits=N_SPLITS)
oof_preds = np.zeros(len(X))
test_preds = np.zeros(len(X_test))

for fold, (train_idx, val_idx) in enumerate(gkf.split(X, y, groups)):
    print(f"--- Fold {fold+1}/{N_SPLITS} ---")
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              eval_metric='auc',
              callbacks=[lgb.early_stopping(100, verbose=False)])

    oof_preds[val_idx] = model.predict_proba(X_val)[:, 1]
    test_preds += model.predict_proba(X_test)[:, 1] / N_SPLITS

oof_score = roc_auc_score(y, oof_preds)
print(f"\nOverall Out-of-Fold AUC Score: {oof_score:.5f}")

# 6. 提出ファイルの作成
print("提出ファイルの作成を開始...")
submission_df['target'] = test_preds
# submission_df.to_csv('submission_baseline.csv', index=False)
display(submission_df)
print("提出ファイル 'submission_baseline.csv' を作成しました。")

特徴量の作成を開始...
データのマージを開始...
データのマージ完了。
前処理を開始...
前処理完了。
モデル学習を開始...
--- Fold 1/5 ---
--- Fold 2/5 ---
--- Fold 3/5 ---
--- Fold 4/5 ---
--- Fold 5/5 ---

Overall Out-of-Fold AUC Score: 0.56146
提出ファイルの作成を開始...


,target
0,0.031189
1,0.038612
2,0.041121
3,0.045208
4,0.035253
...,...
11017,0.056994
11018,0.057746
11019,0.052930
11020,0.069178


提出ファイル 'submission_baseline.csv' を作成しました。


baseline model : 0.56146
